In [1]:
import requests
import pandas as pd

# ✅ Corrected CVE API Endpoint
CVE_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# Set headers to properly request data
headers = {"Accept": "application/json"}

# Fetch CVE Data
response = requests.get(CVE_API_URL, headers=headers)

# Check if API request was successful
if response.status_code == 200 and response.text.strip():
    try:
        cve_data = response.json()

        # Extract Relevant Information
        cve_list = []
        for item in cve_data.get('vulnerabilities', []):
            cve_id = item['cve']['id']
            description = item['cve']['descriptions'][0]['value']
            severity = item['cve'].get('metrics', {}).get('cvssMetricV31', [{}])[0].get('baseSeverity', 'Unknown')

            cve_list.append({"CVE_ID": cve_id, "Description": description, "Severity": severity})

        # Convert to DataFrame for AI Training
        cve_df = pd.DataFrame(cve_list)
        cve_df.to_csv("cve_data.csv", index=False)
        print("✅ CVE Data Successfully Fetched & Saved!")

    except requests.exceptions.JSONDecodeError:
        print("⚠️ Error: Failed to decode JSON response!")
else:
    print(f"⚠️ Error: Received unexpected response ({response.status_code})")

✅ CVE Data Successfully Fetched & Saved!


In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

# Load the CVE dataset
df = pd.read_csv("expanded_mitre_cve_data.csv")

# Drop missing values
df.dropna(inplace=True)

# Convert severity levels to numerical values
severity_mapping = {
    "Critical": 4,
    "High": 3,
    "Medium": 2,
    "Low": 1,
    "Unknown": 0
}
df["Severity_Level"] = df["Severity"].map(severity_mapping)

# Extract keywords from descriptions for AI model training
def extract_keywords(description):
    words = re.findall(r'\b\w+\b', description.lower())  # Tokenize words
    important_words = [word for word in words if len(word) > 3]  # Keep meaningful words
    return " ".join(important_words)

df["Processed_Description"] = df["Description"].apply(extract_keywords)

# Save the preprocessed data
df.to_csv("processed_cve_data.csv", index=False)
print("✅ CVE Data Successfully Preprocessed & Saved!")

✅ CVE Data Successfully Preprocessed & Saved!


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load preprocessed CVE dataset
df = pd.read_csv("expanded_mitre_cve_data.csv")

# Convert severity labels to numerical values
severity_mapping = {"Critical": 4, "High": 3, "Medium": 2, "Low": 1, "Unknown": 0}
df["Severity_Level"] = df["Severity"].map(severity_mapping)

# Select features and labels
X = df.drop(columns=["CVE_ID", "Description", "Severity"])
y = df["Severity_Level"]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
print("Unique Severity Values:", df["Severity_Level"].unique())  # Should be 0, 1, 2, 3, 4

Unique Severity Values: [nan  0.]


In [13]:
df["Severity_Level"] = df["Severity_Level"].fillna(0).astype(int)

In [14]:
df["Severity_Level"] = df["Severity_Level"].apply(lambda x: min(max(x, 0), 4))  # Clamp values to [0, 4]

In [15]:
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

/usr/local/lib/python3.11/dist-packages/pandas/core/series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


In [20]:
# Load preprocessed CVE dataset
df = pd.read_csv("expanded_mitre_cve_data.csv")

# Convert severity labels to numerical values
severity_mapping = {"Critical": 4, "High": 3, "Medium": 2, "Low": 1, "Unknown": 0}
df["Severity_Level"] = df["Severity"].map(severity_mapping)

# Ensure Severity_Level is within the expected range and of the correct type
df["Severity_Level"] = df["Severity_Level"].fillna(0).astype(int)
df["Severity_Level"] = df["Severity_Level"].apply(lambda x: min(max(x, 0), 4))

# Select features and labels
X = df.drop(columns=["CVE_ID", "Description", "Severity"])
y = df["Severity_Level"]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset AFTER preprocessing Severity_Level
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert y_train and y_test to the correct data type AFTER the split
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

# Define AI model structure
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Corrected input layer
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # Multi-class classification (0-4)
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train AI model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

print("✅ AI Model Training Complete!")

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 1.0000 - loss: 1.5896 - val_accuracy: 1.0000 - val_loss: 1.5304
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 1.5111 - val_accuracy: 1.0000 - val_loss: 1.4539
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.4352 - val_accuracy: 1.0000 - val_loss: 1.3799
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.3619 - val_accuracy: 1.0000 - val_loss: 1.3086
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.2912 - val_accuracy: 1.0000 - val_loss: 1.2400
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.2234 - val_accuracy: 1.0000 - val_loss: 1.1742
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.1583 - val_accuracy: 1.0000 - val_loss: 1.1112
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.0960 - val_accuracy: 1.0000 - val_loss

In [17]:
# Evaluate model accuracy
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Model Accuracy: {accuracy:.2f}")
print(f"📉 Model Loss: {loss:.4f}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.1327 
✅ Model Accuracy: 1.00
📉 Model Loss: 0.1327


In [18]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf

# ✅ Updated CVE API URLs
CVE_DETAILS_URL = "https://www.cvedetails.com/"
CVE_MITRE_URL = "https://cve.mitre.org/"
CVE_ORG_URL = "https://www.cve.org/"

# Fetch CVE Data (Placeholder for actual API parsing)
cve_list = [
    {"CVE_ID": "CVE-2025-30073", "Description": "Issue in OPC cardsystems Webapp Aufwertung 2.1.0", "Severity": "High"},
    {"CVE_ID": "CVE-2025-28361", "Description": "Unauthorized stack overflow vulnerability in Telesquare TLR-2005KSH", "Severity": "High"},
    {"CVE_ID": "CVE-2025-26011", "Description": "Telesquare TLR-2005KSH allows unauthorized password modification", "Severity": "Critical"}
]

# Convert to DataFrame
cve_df = pd.DataFrame(cve_list)
cve_df.to_csv("latest_cve_data.csv", index=False)

print("✅ Latest CVE Data Saved!")

# Load the latest CVE data for AI testing
df = pd.read_csv("latest_cve_data.csv")

# Convert severity labels into numerical values
severity_mapping = {"Critical": 4, "High": 3, "Medium": 2, "Low": 1, "Unknown": 0}
df["Severity_Level"] = df["Severity"].map(severity_mapping)

# Select a new CVE sample for AI testing
# Instead of using the "Severity" column directly, use the "Severity_Level" column
new_sample = np.array([df.iloc[0, 3]])  # Pick a sample CVE data point using Severity_Level

# Reshape the new_sample to a 2D array with a single row and the correct number of features
new_sample = new_sample.reshape(1, -1)

# Use StandardScaler to transform the new_sample using the same scaling as the training data
new_sample = scaler.transform(new_sample)

prediction = model.predict(new_sample)
predicted_severity = np.argmax(prediction)

severity_labels = ["Unknown", "Low", "Medium", "High", "Critical"]
print(f"🔒 Predicted Severity: {severity_labels[predicted_severity]}")

✅ Latest CVE Data Saved!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


🔒 Predicted Severity: Unknown


In [19]:
# Modify AI Model to Improve Accuracy & Prevent Overfitting
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Define Optimized AI Model
model = tf.keras.Sequential([
    Input(shape=(X_train.shape[1],)),  # Improved Input layer
    Dense(128, activation='relu'),
    Dropout(0.3),  # Dropout prevents overfitting
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # Multi-class classification (0-4)
])

# Use Adam optimizer with tuned learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train AI Model with Latest CVE Data
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

print("✅ AI Model Fine-Tuning Complete!")

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 1.0000 - loss: 1.5995 - val_accuracy: 1.0000 - val_loss: 1.5697
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.5599 - val_accuracy: 1.0000 - val_loss: 1.5304
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.5208 - val_accuracy: 1.0000 - val_loss: 1.4918
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.4823 - val_accuracy: 1.0000 - val_loss: 1.4539
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.4445 - val_accuracy: 1.0000 - val_loss: 1.4165
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.4074 - val_accuracy: 1.0000 - val_loss: 1.3799
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.3709 - val_accuracy: 1.0000 - val_loss: 1.3439
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.3350 - val_accuracy: 1.0000 - val_loss

In [ ]:
# import requests
# import pandas as pd
# import numpy as np
# import tensorflow as tf

# # ✅ CVE Details & NVD API URLs
# CVE_DETAILS_URL = "https://www.cvedetails.com/"
# NVD_API_URL = "https://www.nist.gov/itl/nvd"

# # Fetch CVE Data (Placeholder for actual API parsing)
# cve_list = [
#     {"CVE_ID": "CVE-2025-32728", "Description": "OpenSSH vulnerability affecting remote authentication", "Severity": "Medium"},
#     {"CVE_ID": "CVE-2025-32381", "Description": "Python xgrammar security flaw leading to unauthorized execution", "Severity": "High"},
#     {"CVE_ID": "CVE-2025-32380", "Description": "Rust Apollo Router misconfiguration exposing sensitive data", "Severity": "Critical"}
# ]

# # Convert to DataFrame
# cve_df = pd.DataFrame(cve_list)
# cve_df.to_csv("cve_details_nvd_data.csv", index=False)

# print("✅ CVE Details & NVD Data Saved!")

# # Load the latest CVE data for AI testing
# df = pd.read_csv("cve_details_nvd_data.csv")

# # Convert severity labels into numerical values
# severity_mapping = {"Critical": 4, "High": 3, "Medium": 2, "Low": 1, "Unknown": 0}
# df["Severity_Level"] = df["Severity"].map(severity_mapping)

# # Select a new CVE sample for AI testing
# # Instead of using the "Severity" column directly, use the "Severity_Level" column
# new_sample = np.array([df.iloc[0, 3]])  # Pick a sample CVE data point using Severity_Level

# # Reshape the new_sample to a 2D array with a single row and one column
# new_sample = new_sample.reshape(1, -1)

# # Use StandardScaler to transform the new_sample using the same scaling as the training data
# # Reshape to match the expected input shape of the StandardScaler
# new_sample_scaled = scaler.transform(new_sample.reshape(1, -1))

# # Make prediction
# prediction = model.predict(new_sample_scaled)
# predicted_severity = np.argmax(prediction)

# severity_labels = ["Unknown", "Low", "Medium", "High", "Critical"]
# print(f"🔒 Predicted Severity: {severity_labels[predicted_severity]}")

In [6]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import os

# NVD API Endpoint
NVD_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# Fetch CVE Data from NVD
headers = {"Accept": "application/json"}
response = requests.get(NVD_API_URL, headers=headers)

if response.status_code == 200:
    cve_data = response.json()
    print("✅ NVD CVE Data Fetched Successfully!")

    # Extract Relevant Information
    cve_list = []
    for item in cve_data.get("vulnerabilities", []):
        cve_id = item['cve']['id']
        description = item['cve']['descriptions'][0]['value']
        severity = item['cve'].get('metrics', {}).get('cvssMetricV2', [{}])[0].get('baseSeverity', 'Unknown')

        cve_list.append({"CVE_ID": cve_id, "Description": description, "Severity": severity})

    # Convert to DataFrame
    cve_df = pd.DataFrame(cve_list)
    cve_df.to_csv("nvd_cve_data.csv", index=False)  # Save to a different file
    print("✅ NVD CVE Data Saved!")

else:
    print(f"⚠️ Error: Unable to fetch data ({response.status_code})")
    print(response.text)  # Print response content for debugging

✅ NVD CVE Data Fetched Successfully!
✅ NVD CVE Data Saved!


In [5]:
import requests
import pandas as pd

# ✅ NVD API Endpoint (MITRE CVE Records)
NVD_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# Fetch CVE Data
headers = {"Accept": "application/json"}
response = requests.get(NVD_API_URL, headers=headers)

if response.status_code == 200:
    cve_data = response.json()
    print("✅ NVD CVE Data Fetched Successfully!")

    # Extract Relevant Information
    cve_list = []
    for item in cve_data.get("vulnerabilities", []):
        cve_id = item["cve"]["id"]
        description = item["cve"]["descriptions"][0]["value"]
        severity = item["cve"].get("metrics", {}).get("cvssMetricV2", [{}])[0].get("baseSeverity", "Unknown")

        cve_list.append({"CVE_ID": cve_id, "Description": description, "Severity": severity})

    # Convert to DataFrame & Save
    cve_df = pd.DataFrame(cve_list)
    cve_df.to_csv("expanded_mitre_cve_data.csv", index=False)
    print("✅ Expanded MITRE CVE Data Saved!")

else:
    print(f"⚠️ Error: Unable to fetch data ({response.status_code})")

✅ NVD CVE Data Fetched Successfully!
✅ Expanded MITRE CVE Data Saved!


In [4]:
# Modify AI Model to Improve Accuracy & Prevent Overfitting
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Define Optimized AI Model
model = tf.keras.Sequential([
    Input(shape=(X_train.shape[1],)),  # Improved Input layer
    Dense(128, activation='relu'),
    Dropout(0.3),  # Dropout prevents overfitting
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # Multi-class classification (0-4)
])

# Use Adam optimizer with tuned learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train AI Model with Latest CVE Data
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

print("✅ AI Model Fine-Tuning Complete!")

NameError: name 'X_train' is not defined

In [3]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import os

# NVD API Endpoint
NVD_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# Fetch CVE Data from NVD
headers = {"Accept": "application/json"}
response = requests.get(NVD_API_URL, headers=headers)

if response.status_code == 200:
    cve_data = response.json()
    print("✅ NVD CVE Data Fetched Successfully!")

    # Extract Relevant Information
    cve_list = []
    for item in cve_data.get("vulnerabilities", []):
        cve_id = item['cve']['id']
        description = item['cve']['descriptions'][0]['value']
        severity = item['cve'].get('metrics', {}).get('cvssMetricV2', [{}])[0].get('baseSeverity', 'Unknown')

        cve_list.append({"CVE_ID": cve_id, "Description": description, "Severity": severity})

    # Convert to DataFrame
    cve_df = pd.DataFrame(cve_list)
    cve_df.to_csv("nvd_cve_data.csv", index=False)  # Save to a different file
    print("✅ NVD CVE Data Saved!")

else:
    print(f"⚠ Error: Unable to fetch data ({response.status_code})")
    print(response.text)  # Print response content for debugging

✅ NVD CVE Data Fetched Successfully!
✅ NVD CVE Data Saved!


In [ ]:
!git clone https://github.com/tanyawadhera08/your-repo.git

Cloning into 'your-repo'...
fatal: could not read Username for 'https://github.com': No such device or address
